In [164]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import string

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [165]:
data=pd.read_csv("ratings_and_sentiments.csv",encoding= 'unicode_escape')

In [166]:
pd.set_option('display.max_colwidth', -1)

In [167]:
data=data[['review_text','num_rating']]
data.head()

review_text  num_rating
0   11/25/2016 1 check-in Love love loved the vibe! Every corner of the coffee shop had its own style, and there were swings!!! I ordered the matcha coffee, and it was muy fantastico! Ordering and getting my drink were pretty streamlined. I ordered on an iPad, which included all beverage selections that ranged from coffee to alcohol, desired level of sweetness, and a checkout system. I got my coffee within minutes!  I was hoping for a typical heart or feather on my coffee, but found myself listing out all the possibilities of what the vibe may be. Any ideas?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        5.0       
1   12/2/2016 Listed in Date Night: Austin, vibe in Austin BEAUTIFUL!!!! Love the vibe! Instagram-worthy!!!  Definitely $$$, so be prepared. This is gonna cost you a pretty penny :) food food was just decent...nothing to rave about.  But, will probably be back just to be somewhere unique and nice.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  4.0       
2   11/30/2016 1 check-in Listed in food seating I loved the eclectic and homey plush vibe and who doesn't want to swing and drink their coffee?  I would categorize this as a pricier coffee place but, to me it was worth it.  After Thanksgiving nobody wants to make food so we headed out in search of food and foods.  There is ample parking in the private lot and  ordering is done through I-Pads.  Pick a seating and they will call your name when your items are ready.  Be patient because some of the coffee drinks take longer than others.  I had the cold coffee coffee, food and foods and we shared a pop+art!  The po+art was strawberry; light and fluffy on the outside and gooey sweet filling on the inside.  The food and foods wasn't the standard so expect different, complex and interesting flavors from the accompanying slaw and sauce.  I thoroughly enjoyed it but, the food was a bit bland and maybe could have used a bit of honey to spruce it up.  The coffee here is spot on and I hear that their hot chocolate is the bomb but, I will have to wait for the weather to cool off to try that.  A great place to be vibe and chat with your friends, a date or read a book. food and foods Pop tart Coffee See all photos from parkingmela S. for The Factory - Cafe With a Soul   4.0       
3   11/25/2016 Very cool vibe! Good drinks Nice seating  However...  Just about everything is bad price. $1.50 extr

In [168]:
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text
lemmatizer = WordNetLemmatizer()


In [169]:
def process_text(text):

    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    text = re.sub(r'\$\w*', '', text)
    # remove old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', text)
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    # remove hashtags
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', text)
    text = str(re.sub("\S*\d\S*", "", text).strip()) 
    text=decontract(text)
    # tokenize texts
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tokens = tokenizer.tokenize(text)

    texts_clean = []
    for word in tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation+'...'):  # remove punctuation
            # 
            stem_word = lemmatizer.lemmatize(word,"v")  # Lemmatizing word
            texts_clean.append(stem_word)

    return " ".join(texts_clean)

In [170]:
data['review_text']=data['review_text'].apply(lambda x: process_text(str(x)))

In [171]:
data=data.loc[(data['num_rating']>3) | (data['num_rating']<3)]
data['num_rating']=data['num_rating'].apply(lambda x: 1 if (x>3)  else 0)
data=data.reset_index()
data=data[['review_text','num_rating']]
data.head()

review_text  num_rating
0  check-in love love love vibe every corner coffee shop style swing order matcha coffee muy fantastico order get drink pretty streamline order ipad include beverage selections range coffee alcohol desire level sweetness checkout system get coffee within minutes hop typical heart feather coffee find list possibilities vibe may ideas                                                                                                                                                                                                                                                                                                                                                                                                     1         
1  list date night austin vibe austin beautiful love vibe instagram-worthy definitely prepare gonna cost pretty penny :) food food decent nothing rave probably back somewhere unique nice                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         1         
2  check-in list food seat love eclectic homey plush vibe want swing drink coffee would categorize pricier coffee place worth thanksgiving nobody want make food head search food foods ample park private lot order do i-pads pick seat call name items ready patient coffee drink take longer others cold coffee coffee food foods share pop art po art strawberry light fluffy outside gooey sweet fill inside food foods standard expect different complex interest flavor accompany slaw sauce thoroughly enjoy food bite bland maybe could use bite honey spruce coffee spot hear hot chocolate bomb wait weather cool try great place vibe chat friends date read book food foods pop tart coffee see photos parkingmela factory cafe soul  1         
3  cool vibe good drink nice seat however everything bad price extra ounces almond milk coffee wifi vibe bite loud mix odd pleasant french oldies follow loud techno several seat dirty get service average feel like los angeles coffee shoot place                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               0         
4  check-in location within northcross mall shop center face east toward burnet rd plenty park love computerize user friendly order system make easy pick choose items want try order pop tart food foods strawberry foods photos upload place ideal order dish share group seat limit since place fairly new lot visitors arrive early prepare wait bite large grop store front face burnet rd see photos michelle factory cafe soul                                                                                                                                                                                                                                                                                                              1

In [172]:
X_train, X_test, y_train, y_test = train_test_split(data['review_text'], data['num_rating'], test_size=0.2,random_state=42)


In [173]:
top_words = 6000
tokenizer = Tokenizer(num_words=top_words)
tokenizer.fit_on_texts(X_train)
list_tokenized_train = tokenizer.texts_to_sequences(X_train)

max_review_length = 130
X_train = pad_sequences(list_tokenized_train, maxlen=max_review_length)


In [174]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words+1, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 130, 32)           192032    
                                                                 
 lstm_3 (LSTM)               (None, 100)               53200     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 245,333
Trainable params: 245,333
Non-trainable params: 0
_________________________________________________________________


In [175]:
model.fit(X_train,y_train, epochs=2, batch_size=64)


Epoch 1/2
86/86 [==============================] - 16s 158ms/step - loss: 0.3738 - accuracy: 0.8880
Epoch 2/2
86/86 [==============================] - 13s 156ms/step - loss: 0.2125 - accuracy: 0.9222


In [176]:
list_tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(list_tokenized_test, maxlen=max_review_length)
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

Accuracy of the model :  0.9295058139534884
F1-score:  0.961246504194966
Confusion matrix:


array([[  76,   75],
       [  22, 1203]])

In [177]:
pip install lime

In [178]:
from lime.lime_text import LimeTextExplainer
class_names=['negative','positive']
explainer= LimeTextExplainer(class_names=class_names)
def predict_proba(arr):
  processed=[]
  for i in arr:
    processed.append(process_text(i))
  list_tokenized_ex = tokenizer.texts_to_sequences(processed)
  Ex = pad_sequences(list_tokenized_ex, maxlen=max_review_length)
  pred=model.predict(Ex)
  returnable=[]
  for i in pred:
    temp=i[0]
    returnable.append(np.array([1-temp,temp])) #I would recommend rounding temp and 1-temp off to 2 places
  return np.array(returnable)




In [179]:
data_to_test=pd.read_csv("ratings_and_sentiments.csv",encoding= 'unicode_escape')[['review_text','num_rating']]

In [200]:
print("Actual rating",data_to_test['num_rating'][5])
explainer.explain_instance(data_to_test['review_text'][5],predict_proba).show_in_notebook(text=True)

Actual rating 4.0


In [199]:
print("Actual rating",data_to_test['num_rating'][3])

explainer.explain_instance(data_to_test['review_text'][3],predict_proba).show_in_notebook(text=True)

Actual rating 2.0


In [197]:
print("Actual rating",data_to_test['num_rating'][7614])
explainer.explain_instance(data_to_test['review_text'][7614],predict_proba).show_in_notebook(text=True)

Actual rating 3.0


In [198]:
print("Actual rating",data_to_test['num_rating'][7574])

explainer.explain_instance(data_to_test['review_text'][7574],predict_proba).show_in_notebook(text=True)

Actual rating 2.0


In [192]:
data_to_test['review_text'].loc[data_to_test['num_rating']<3]

3        11/25/2016 Very cool vibe! Good drinks Nice seating  However...  Just about everything is bad price. $1.50 extra for 3 ounces of Almond Milk in a coffee. No WiFi vibe is a bit loud and the mix is odd.  Pleasant French oldies followed by loud techno. Several seatings were dirty when we got there. Service is average.  It feels like a Los Angeles coffee shot that is out of place.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    